In [39]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import sys
from scipy.optimize import brentq, least_squares
from scipy.stats import norm
from scipy.interpolate import RegularGridInterpolator
import scipy.integrate as integrate
from scipy import interpolate

sys.path.append("..")

# handy it seems
# https://docs.sympy.org/latest/modules/solvers/solvers.html
from sympy.solvers import solve
from sympy import Symbol
from analytical_option_formulae.option_types.vanilla_option import VanillaOption


# read data
df_sabr_alpha = pd.read_csv("../swaption_calibration/sabr_alpha.csv")
df_sabr_beta = pd.read_csv("../swaption_calibration/sabr_beta.csv")
df_sabr_nu = pd.read_csv("../swaption_calibration/sabr_nu.csv")
df_sabr_rho = pd.read_csv("../swaption_calibration/sabr_rho.csv")

df_combined = pd.read_csv("../bootstrap_swap_curve/df_combined.csv")

In [40]:
# adjust sabr param df
def adjust_sabr_params(my_sabr_df):
    my_sabr_df.set_index("expiry", inplace=True)
    my_sabr_df.columns.name = "tenor"
    my_sabr_df.columns = my_sabr_df.columns.astype(float)

In [41]:
adjust_sabr_params(df_sabr_alpha)
adjust_sabr_params(df_sabr_beta)
adjust_sabr_params(df_sabr_nu)
adjust_sabr_params(df_sabr_rho)

In [42]:
print(df_sabr_alpha.index)
print(df_sabr_alpha.columns)
print(df_sabr_alpha.values.T)

Float64Index([1.0, 5.0, 10.0], dtype='float64', name='expiry')
Float64Index([1.0, 2.0, 3.0, 5.0, 10.0], dtype='float64', name='tenor')
[[0.13907382 0.16661823 0.17824663]
 [0.1846499  0.1995335  0.19618447]
 [0.19685208 0.21030546 0.2077691 ]
 [0.17804234 0.19097227 0.19916746]
 [0.17114286 0.17705111 0.17634725]]


In [43]:
# setup sabr interpolators
sabr_alpha_interp = interpolate.interp2d(
    df_sabr_alpha.index, df_sabr_alpha.columns, df_sabr_alpha.values.T, kind="linear"
)
sabr_beta_interp = interpolate.interp2d(
    df_sabr_beta.index, df_sabr_beta.columns, df_sabr_beta.values.T, kind="linear"
)
sabr_rho_interp = interpolate.interp2d(
    df_sabr_rho.index, df_sabr_rho.columns, df_sabr_rho.values.T, kind="linear"
)
sabr_nu_interp = interpolate.interp2d(
    df_sabr_nu.index, df_sabr_nu.columns, df_sabr_nu.values.T, kind="linear"
)

C:\Users\kilo\AppData\Local\Temp\ipykernel_19748\2151748497.py:2: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  sabr_alpha_interp = interpolate.interp2d(
C:\Users\kilo\AppData\Local\Temp\ipykernel_19748\2151748497.py:5: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regula

In [44]:
# expand df_combined
combined_tenors = np.arange(0, 30.25, 0.25)
df_combined_new = pd.DataFrame({"tenor": combined_tenors})
df_combined_new = pd.merge(df_combined_new, df_combined, how="left", on="tenor")
df_combined_new.loc[0, "ois_df"] = 1
df_combined_new.loc[0, "irs_df"] = 1
df_combined_new.loc[0, "fw_libor"] = 0
df_combined_new.loc[0, "irs_rate"] = 0
df_combined_new.interpolate(inplace=True)

In [45]:
df_combined_new

,tenor,ois_df,irs_rate,fw_libor,irs_df
0,0.00,1.000000,0.000000,0.000000,1.000000
1,0.25,0.999376,0.012500,0.012500,0.993827
2,0.50,0.998752,0.025000,0.025000,0.987654
3,0.75,0.997880,0.026500,0.028003,0.980116
4,1.00,0.997009,0.028000,0.031005,0.972577
...,...,...,...,...,...
116,29.00,0.852674,0.049500,0.076225,0.242118
117,29.25,0.851357,0.049625,0.077735,0.237504
118,29.50,0.850040,0.049750,0.079246,0.232890
119,29.75,0.848723,0.049875,0.080880,0.228276


Using the SABR model calibrated in the previous question, value the
following constant maturity swap (CMS) products:

* PV of a leg receiving CMS10y semi-annually over the next 5 years
* PV of a leg receiving CMS2y quarterly over the next 10 years

Setup with Prof Tee's code

In [46]:
class AbstractBlack76Model:
    """
    A base class used to model Black-Scholes option model
    ...
    Parameters
    ----------
    F : float
        The forward price of the underlying asset
    K : float
        The strike price of the options
    discount_factor : float
        The "numeraire" discount factor of the model (i.e. PVBP, compounded discount factor)
    sigma : float
        Volatility
    T : float
        Maturity period (years)
    """

    def __init__(
        self,
        F: float,
        K: float,
        discount_factor: float,
        sigma: float,
        T: float,
    ):
        self.F = F
        self.K = K
        self.sigma = sigma
        self.T = T

        self.d1 = self._calculate_d1()
        self.d2 = self._calculate_d2()
        self.discount_factor = discount_factor

    def _calculate_d1(self) -> float:
        return (np.log(self.F / self.K) + self.sigma**2 / 2 * self.T) / (
            self.sigma * np.sqrt(self.T)
        )

    def _calculate_d2(self) -> float:
        return self.d1 - self.sigma * np.sqrt(self.T)


class VanillaBlack76Model(AbstractBlack76Model):
    def calculate_call_price(self) -> float:
        return self.discount_factor * (
            self.F * norm.cdf(self.d1) - self.K * norm.cdf(self.d2)
        )

    def calculate_put_price(self) -> float:
        return self.discount_factor * (
            -self.F * norm.cdf(-self.d1) + self.K * norm.cdf(-self.d2)
        )


class AbstractDisplacedDiffusionModel:
    """
    Displaced diffusion is extension of Black76 with an additional parameter beta
    ...
    Parameters
    ----------
    F : float
        The forward price of the underlying asset
    K : float
        The strike price of the options
    discount_factor : float
        The "numeraire" discount factor of the model (i.e. PVBP, compounded discount factor)
    sigma : float
        Volatility
    T : float
        Maturity period (years)
    beta : float
        Displaced diffusion model parameter (0,1], but lecture notes say [0,1]
        https://ink.library.smu.edu.sg/cgi/viewcontent.cgi?article=6976&context=lkcsb_research
    """

    def __init__(
        self,
        F: float,
        K: float,
        discount_factor: float,
        sigma: float,
        T: float,
        beta: float,
    ):
        self.F = F
        self.K = K
        self.sigma = sigma
        self.T = T
        self.beta = beta

        self.adjusted_F = self.F / self.beta
        self.adjusted_K = self.K + ((1 - self.beta) / self.beta) * self.F
        self.adjusted_sigma = self.sigma * self.beta
        self.discount_factor = discount_factor

        self.d1 = self._calculate_d1()
        self.d2 = self._calculate_d2()

    def _calculate_d1(self) -> float:
        return (
            np.log(self.adjusted_F / self.adjusted_K)
            + 0.5 * self.adjusted_sigma**2 * self.T
        ) / (self.adjusted_sigma * np.sqrt(self.T))

    def _calculate_d2(self) -> float:
        return self.d1 - self.adjusted_sigma * np.sqrt(self.T)


class VanillaDisplacedDiffusionModel(AbstractDisplacedDiffusionModel):
    def calculate_call_price(self) -> float:
        return self.discount_factor * (
            self.adjusted_F * norm.cdf(self.d1) - self.adjusted_K * norm.cdf(self.d2)
        )

    def calculate_put_price(self) -> float:
        return self.discount_factor * (
            self.adjusted_K * norm.cdf(-self.d2) - self.adjusted_F * norm.cdf(-self.d1)
        )


class VanillaOption:
    def black_model(
        self, F: float, K: float, discount_factor: float, sigma: float, T: float
    ) -> VanillaBlack76Model:
        return VanillaBlack76Model(F, K, discount_factor, sigma, T)

    def displaced_diffusion_model(
        self,
        F: float,
        K: float,
        discount_factor: float,
        sigma: float,
        T: float,
        beta: float,
    ) -> AbstractDisplacedDiffusionModel:
        return VanillaDisplacedDiffusionModel(F, K, discount_factor, sigma, T, beta)

In [47]:
# SABR
def SABR_model(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta) ** 2) / 24) * alpha * alpha / (F ** (2 - 2 * beta))
        numer2 = 0.25 * rho * beta * nu * alpha / (F ** (1 - beta))
        numer3 = ((2 - 3 * rho * rho) / 24) * nu * nu
        VolAtm = alpha * (1 + (numer1 + numer2 + numer3) * T) / (F ** (1 - beta))
        sabrsigma = VolAtm
    else:
        z = (nu / alpha) * ((F * X) ** (0.5 * (1 - beta))) * np.log(F / X)
        zhi = np.log((((1 - 2 * rho * z + z * z) ** 0.5) + z - rho) / (1 - rho))
        numer1 = (((1 - beta) ** 2) / 24) * ((alpha * alpha) / ((F * X) ** (1 - beta)))
        numer2 = 0.25 * rho * beta * nu * alpha / ((F * X) ** ((1 - beta) / 2))
        numer3 = ((2 - 3 * rho * rho) / 24) * nu * nu
        numer = alpha * (1 + (numer1 + numer2 + numer3) * T) * z
        denom1 = ((1 - beta) ** 2 / 24) * (np.log(F / X)) ** 2
        denom2 = (((1 - beta) ** 4) / 1920) * ((np.log(F / X)) ** 4)
        denom = ((F * X) ** ((1 - beta) / 2)) * (1 + denom1 + denom2) * zhi
        sabrsigma = numer / denom

    return sabrsigma


def calculate_SABR_vol_err(x, strikes, vols, F, T, beta):
    err = 0.0
    for i, vol in enumerate(vols):
        err += (vol - SABR_model(F, strikes[i], T, x[0], beta, x[1], x[2])) ** 2
    return err

In [48]:
# setup vanilla option
vanilla_option = VanillaOption()

In [49]:
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1 / K * (1.0 - 1 / (1 + K / m) ** (N * m))
    return value


def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1 / K * IRR_0(K, m, N) + 1 / (K * m) * N * m / (1 + K / m) ** (
        N * m + 1
    )
    return firstDerivative


def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2 / K * IRR_1(K, m, N) - 1 / (K * m * m) * (N * m) * (
        N * m + 1
    ) / (1 + K / m) ** (N * m + 2)
    return secondDerivative

In [50]:
def g_0(K):
    return K


def g_1(K):
    return 1.0


def g_2(K):
    return 0.0

In [51]:
def h_0(K, m, N):
    # implementation of h(K)
    value = g_0(K) / IRR_0(K, m, N)
    return value


def h_1(K, m, N):
    # implementation of h'(K) (1st derivative)
    firstDerivative = (IRR_0(K, m, N) * g_1(K) - g_0(K) * IRR_1(K, m, N)) / IRR_0(
        K, m, N
    ) ** 2
    return firstDerivative


def h_2(K, m, N):
    # implementation of h''(K) (2nd derivative)
    secondDerivative = (
        IRR_0(K, m, N) * g_2(K)
        - IRR_2(K, m, N) * g_0(K)
        - 2.0 * IRR_1(K, m, N) * g_1(K)
    ) / IRR_0(K, m, N) ** 2 + 2.0 * IRR_1(K, m, N) ** 2 * g_0(K) / IRR_0(K, m, N) ** 3
    return secondDerivative

In [52]:
def v_payer(discount_factor, F, K, sigma, expiry, payment_freq, tenor):
    b76_model = vanilla_option.black_model(F, K, discount_factor, sigma, expiry)
    return (
        discount_factor
        * IRR_0(F, payment_freq, tenor)
        * b76_model.calculate_call_price()
    )


def v_receiver(discount_factor, F, K, sigma, expiry, payment_freq, tenor):
    b76_model = vanilla_option.black_model(F, K, discount_factor, sigma, expiry)
    return (
        discount_factor
        * IRR_0(F, payment_freq, tenor)
        * b76_model.calculate_put_price()
    )

In [53]:
def integ_receiver_func(
    K, discount_factor, F, alpha, beta, nu, rho, expiry, payment_freq, tenor
):
    sabr_sigma = SABR_model(
        F,
        K,
        expiry,
        alpha,
        beta,
        rho,
        nu,
    )
    return h_2(K, payment_freq, tenor) * v_receiver(
        discount_factor, F, K, sabr_sigma, expiry, payment_freq, tenor
    )


def integ_payer_func(
    K, discount_factor, F, alpha, beta, nu, rho, expiry, payment_freq, tenor
):
    sabr_sigma = SABR_model(
        F,
        K,
        expiry,
        alpha,
        beta,
        rho,
        nu,
    )
    return h_2(K, payment_freq, tenor) * v_payer(
        discount_factor, F, K, sabr_sigma, expiry, payment_freq, tenor
    )

In [54]:
def get_pvbp_pvfloat_krate(my_dataframe, payment_freq):
    divisor = 1 / payment_freq
    for x, row in my_dataframe.iterrows():
        expiry = my_dataframe.loc[x, "expiry"]
        tenor = my_dataframe.loc[x, "tenor"]
        time_sum = expiry + tenor
        tenor_range = np.arange(expiry, time_sum + divisor, divisor)
        rows_to_work = df_combined_new[df_combined_new["tenor"].isin(tenor_range)]
        rows_to_work = rows_to_work.drop(rows_to_work.index[[0]])
        my_dataframe.loc[x, "pv_fix"] = divisor * rows_to_work["ois_df"].sum()
        my_dataframe.loc[x, "pv_float"] = (
            divisor * (rows_to_work["ois_df"] * rows_to_work["fw_libor"]).sum()
        )
        my_dataframe.loc[x, "s_nn0"] = (
            my_dataframe.loc[x, "pv_float"] / my_dataframe.loc[x, "pv_fix"]
        )

In [55]:
def compute_pv_cms(payment_duration, payment_freq, tenor, df_combined_new):
    # 1. we construct pvbp, pvfloat, snN0 dataset first
    start_time = 1 / payment_freq
    expiry = np.arange(start_time, payment_duration + start_time, start_time)
    temp_df = pd.DataFrame()
    temp_df["expiry"] = expiry
    temp_df["tenor"] = tenor
    temp_df["expiry"] = temp_df["expiry"].astype(float)
    temp_df["tenor"] = temp_df["tenor"].astype(float)
    get_pvbp_pvfloat_krate(temp_df, payment_freq)

    for x, rows in temp_df.iterrows():
        tenor = temp_df.loc[x, "tenor"]
        expiry = temp_df.loc[x, "expiry"]
        alpha = sabr_alpha_interp(expiry, tenor)[0]
        beta = sabr_beta_interp(expiry, tenor)[0]
        nu = sabr_nu_interp(expiry, tenor)[0]
        rho = sabr_rho_interp(expiry, tenor)[0]
        # print(f"for {expiry} x {tenor} = {alpha} {beta} {rho} {nu}")
        temp_df.loc[x, "alpha"] = alpha
        temp_df.loc[x, "beta"] = beta
        temp_df.loc[x, "nu"] = nu
        temp_df.loc[x, "rho"] = rho
        disc_factor = temp_df.loc[x, "pv_fix"]
        # g(F)
        first_term = temp_df.loc[x, "s_nn0"]

        # receiver integ
        second_term = integrate.quad(
            lambda x: integ_receiver_func(
                x,
                disc_factor,
                first_term,
                alpha,
                beta,
                nu,
                rho,
                expiry,
                payment_freq,
                tenor,
            ),
            0,
            first_term,
        )
        second_term = second_term[0] / disc_factor

        # third integ
        third_term = integrate.quad(
            lambda x: integ_payer_func(
                x,
                disc_factor,
                first_term,
                alpha,
                beta,
                nu,
                rho,
                expiry,
                payment_freq,
                tenor,
            ),
            first_term,
            100,
        )
        third_term = third_term[0] / disc_factor

        sum = first_term + second_term + third_term
        temp_df.loc[x, "s_nnT"] = sum

    print(temp_df)

In [56]:
# CMS10y semi-annually over the next 5 years
payment_duration = 5
payment_freq = 2
tenor = 10
pv = compute_pv_cms(payment_duration, payment_freq, tenor, df_combined_new)

C:\Users\kilo\AppData\Local\Temp\ipykernel_19748\3307331637.py:15: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
        scattered 2D data.

        In new code, for regular grids use `RegularGridInterpolator` instead.
        For scattered data, prefer `LinearNDInterpolator` or
        `CloughTocher2DInterpolator`.

        For more details see
        `https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  alpha = sabr_alpha_interp(expiry, tenor)[0]
C:\Users\kilo\AppData\Local\Temp\ipykernel_19748\3307331637.py:16: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

        For legacy code, nearly bug-for-bug compatible replacements are
    

   expiry  tenor    pv_fix  pv_float     s_nn0     alpha  beta        nu  \
0     0.5   10.0  9.771240  0.369790  0.037845  0.171143   0.9  0.777278   
1     1.0   10.0  9.747887  0.374591  0.038428  0.171143   0.9  0.777278   
2     1.5   10.0  9.723986  0.379428  0.039020  0.171881   0.9  0.742302   
3     2.0   10.0  9.699536  0.384435  0.039634  0.172620   0.9  0.707325   
4     2.5   10.0  9.674546  0.388915  0.040200  0.173358   0.9  0.672348   
5     3.0   10.0  9.649018  0.393564  0.040788  0.174097   0.9  0.637372   
6     3.5   10.0  9.623045  0.398507  0.041412  0.174836   0.9  0.602395   
7     4.0   10.0  9.596629  0.403650  0.042062  0.175574   0.9  0.567418   
8     4.5   10.0  9.569778  0.409882  0.042831  0.176313   0.9  0.532442   
9     5.0   10.0  9.542492  0.416374  0.043634  0.177051   0.9  0.497465   

        rho     s_nnT  
0 -0.265908  0.040129  
1 -0.265908  0.044138  
2 -0.287485  0.055965  
3 -0.309062  0.115859  
4 -0.330639  0.273235  
5 -0.352216  0.5028

In [57]:
# PV of a leg receiving CMS2y quarterly over the next 10 years
payment_duration = 10
payment_freq = 4
tenor = 2
pv = compute_pv_cms(payment_duration, payment_freq, tenor, df_combined_new)

C:\Users\kilo\AppData\Local\Temp\ipykernel_19748\3307331637.py:15: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
        scattered 2D data.

        In new code, for regular grids use `RegularGridInterpolator` instead.
        For scattered data, prefer `LinearNDInterpolator` or
        `CloughTocher2DInterpolator`.

        For more details see
        `https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  alpha = sabr_alpha_interp(expiry, tenor)[0]
C:\Users\kilo\AppData\Local\Temp\ipykernel_19748\3307331637.py:16: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

        For legacy code, nearly bug-for-bug compatible replacements are
    

    expiry  tenor    pv_fix  pv_float     s_nn0     alpha  beta        nu  \
0     0.25    2.0  1.991408  0.060890  0.030577  0.184650   0.9  1.677383   
1     0.50    2.0  1.989664  0.063132  0.031730  0.184650   0.9  1.677383   
2     0.75    2.0  1.987917  0.064697  0.032545  0.184650   0.9  1.677383   
3     1.00    2.0  1.986169  0.065586  0.033021  0.184650   0.9  1.677383   
4     1.25    2.0  1.984394  0.066428  0.033475  0.185580   0.9  1.638922   
5     1.50    2.0  1.982593  0.067224  0.033907  0.186510   0.9  1.600460   
6     1.75    2.0  1.980766  0.068033  0.034347  0.187441   0.9  1.561999   
7     2.00    2.0  1.978912  0.068855  0.034794  0.188371   0.9  1.523538   
8     2.25    2.0  1.977032  0.069294  0.035049  0.189301   0.9  1.485077   
9     2.50    2.0  1.975127  0.069350  0.035112  0.190231   0.9  1.446616   
10    2.75    2.0  1.973195  0.069407  0.035175  0.191161   0.9  1.408154   
11    3.00    2.0  1.971237  0.069465  0.035239  0.192092   0.9  1.369693   